<a href="https://colab.research.google.com/github/ManuSinghYadav/Pneumonia-Classification-from-X-Rays/blob/main/Pneumonia_Classification_from_X_Rays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Loading the Dataset & Image Preprocessing
Dataset of Images can be downloaded from [here](https://www.kaggle.com/datasets/khoongweihao/covid19-xray-dataset-train-test-sets).

Loaded the images from the drive after mounting it in our colab.

In [2]:
import tensorflow as tf

Preprocessing of the training dataset.

In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Pneumonia Dataset/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 148 images belonging to 2 classes.


Preprocessing of the testing dataset.

In [5]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Pneumonia Dataset/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 40 images belonging to 2 classes.


###Building & Training our CNN model

In [7]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [9]:
cnn.add(tf.keras.layers.Flatten())

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
5/5 [==============================] - 112s 22s/step - loss: 1.5895 - accuracy: 0.5608 - val_loss: 1.0256 - val_accuracy: 0.5000
Epoch 2/25
5/5 [==============================] - 7s 1s/step - loss: 0.7677 - accuracy: 0.6014 - val_loss: 0.3172 - val_accuracy: 0.9750
Epoch 3/25
5/5 [==============================] - 5s 1s/step - loss: 0.5687 - accuracy: 0.6554 - val_loss: 0.3251 - val_accuracy: 0.9750
Epoch 4/25
5/5 [==============================] - 6s 1s/step - loss: 0.4642 - accuracy: 0.7905 - val_loss: 0.2751 - val_accuracy: 0.9750
Epoch 5/25
5/5 [==============================] - 6s 1s/step - loss: 0.3916 - accuracy: 0.8446 - val_loss: 0.2710 - val_accuracy: 0.9250
Epoch 6/25
5/5 [==============================] - 5s 1s/step - loss: 0.3733 - accuracy: 0.8716 - val_loss: 0.1534 - val_accuracy: 0.9750
Epoch 7/25
5/5 [==============================] - 5s 1s/step - loss: 0.3756 - accuracy: 0.8176 - val_loss: 0.1230 - val_accuracy: 0.9750
Epoch 8/25
5/5 [======================

Our CNN model is now trained and its efficiency can be seen from the validation accuracy, which is around 100%.